# 1. 强化学习概述
典型的强化学习环境如下所示：
![强化学习环境](./images/chp_02_01_001.png)
由上图可以看出，强化学习系统由环境和Agent组成，如下所示：
* 在第$t$时刻，Agent观察到环境状态$S_{t}$；
* 通过计算，Agent选择行动$A_{t}$；
* 由于Agent采取的行动，环境转移到$S_{t+1}$，并给Agent一个奖励信号$R_{t}$；

以上过程循环往复进行，直到过程结束，如游戏结束。
一个过程（如游戏的一局）我们称之为Trajectory，我们用$\tau$表示：
$$
\tau=\{ (r_{0}, s_{1}, a_{1}), (r_{1}, s_{2}, a_{2}), (r_{2}, s_{3}, a_{3}), ..., (r_{T-1}, s_{T}, a_{T}) \}
$$
我们可以计算某个Trajectory出现的概率：
$$
p_{\theta}(\tau)=p(s_{1})p_{\theta}(a_{1} | s_{1})p(s_{2} | s_{1}, a_{1})p_{\theta}(a_{2} | s_{2})... \\
=p(s_{1})\prod_{t=1}^{T} p_{\theta}(a_{t} | s_{t})p(s_{t+1} | s_{t}, a_{t})
$$

我们需要让Agent可以找到累积奖励最大的策略，我们首先定义累积奖励：
$$
R(\tau)=\sum_{t=1}^{T} r_{t}
$$
在实际中，我们以游戏为例，每局都具有随机性，所以得到的累积奖励也会不同，所以我们将我们的目标定为使累积奖励的平均值达到最大，我们定义平均累积奖励：
$$
\bar{R}(\tau)=\sum_{\tau}p_{\theta}(\tau)R(\tau)
$$
我们现在的任务是求这个值的最大值，在数学上，求最大值就是求微分，然后按梯度上升法求解。
$$
\nabla_{\theta} \bar{R}(\tau) = \sum_{\tau} R(\tau) \nabla_{\theta} p_{\theta}(\tau) = \sum_{\tau} R(\tau) p_{\theta}(\tau) \frac{\nabla_{\theta} p_{\theta}(\tau)}{p_{\theta}(\tau)} \\
=\sum_{\tau} R(\tau) p_{\theta}(\tau) \nabla_{\theta} \log p_{\theta}(\tau) \\
=E_{\tau \sim p_{\theta}(\tau)} R(\tau) \nabla_{\theta} \log p_{\theta}(\tau) \\
=\frac{1}{N}\sum_{n=1}^{N} R(\tau ^{n}) \nabla_{\theta} \log p_{\theta}(\tau ^{n}) \\
=\frac{1}{N}\sum_{n=1}^{N} R(\tau ^{n}) \sum_{t=1}^{T_{n}} \nabla_{\theta} \log p_{\theta}(a_{t}^{n} | s_{t}^{n}) \\
=\frac{1}{N}\sum_{n=1}^{N} \sum_{t=1}^{T_{n}} R(\tau ^{n}) \nabla_{\theta} \log p_{\theta}(a_{t}^{n} | s_{t}^{n})
$$
在上面的推导中，我们用到了一个求导公式：
$$
\frac{d(\log x)}{dx}=\frac{1}{x}
$$
根据求导的换元法：
$$
\nabla _{\theta} \log p_{\theta}(\tau) = \frac{ \partial{ \log p_{\theta}(\tau)} }{\partial{\theta}} = \frac{ \partial{ \log p_{\theta}(\tau)} }{ \partial{ p_{\theta}(\tau) } } \frac{{ \partial{ p_{\theta}(\tau) } }}{\partial{ \theta }}=\frac{1}{ p_{\theta}(\tau) } \nabla _{\theta} p_{\theta}(\tau)= \frac{\nabla _{\theta}p_{\theta}(\tau)}{ p_{\theta}(\tau) }  
$$


以上是理认上的分析过程，我们在实际中怎么来操作呢？我们可以按照如下方法操作，我们首先利用Agent与环境产生互动，例如玩$N$局游戏，得到如下$N$个Trajectory：
$$
\tau ^{1}=\{ (r_{0}^{1}, s_{1}^{1}, a_{1}^{1}), (r_{1}^{1}, s_{2}^{1}, a_{2}^{1}), (r_{2}^{1}, s_{3}^{1}, a_{3}^{1}), ..., (r_{T-1}^{1}, s_{T}^{1}, a_{T}^{1}) \}, R^(\tau ^{1}) \\
\tau ^{2}=\{ (r_{0}^{2}, s_{1}^{2}, a_{1}^{2}), (r_{1}^{2}, s_{2}^{2}, a_{2}^{2}), (r_{2}^{2}, s_{3}^{2}, a_{3}^{2}), ..., (r_{T-1}^{2}, s_{T}^{2}, a_{T}^{2}) \}, R(\tau ^{2}) \\
...... \\
\tau ^{n}=\{ (r_{0}^{n}, s_{1}^{n}, a_{1}^{n}), (r_{1}^{n}, s_{2}^{n}, a_{2}^{n}), (r_{2}^{n}, s_{3}^{n}, a_{3}^{n}), ..., (r_{T-1}^{n}, s_{T}^{n}, a_{T}^{n}) \}, R(\tau ^{n}) \\
...... \\
\tau ^{N}=\{ (r_{0}^{N}, s_{1}^{N}, a_{1}^{N}), (r_{1}^{N}, s_{2}^{N}, a_{2}^{N}), (r_{2}^{N}, s_{3}^{N}, a_{3}^{N}), ..., (r_{T-1}^{N}, s_{T}^{N}, a_{T}^{N}) \}, R(\tau ^{N}) 
$$
接下来我们上述$N$个Trajectory，变为监督学习的数据集形式：
$$
(s_{1}^{1}, a_{1}^{1}) \oplus R(\tau ^{1}) \quad (s_{2}^{1}, a_{2}^{1}) \oplus R(\tau ^{1}) \quad (s_{3}^{1}, a_{3}^{1}) \oplus R(\tau ^{1}) \quad ... \quad (s_{T}^{1}, a_{T}^{1}) \oplus R(\tau ^{1}) \\
(s_{1}^{2}, a_{1}^{2}) \oplus R(\tau ^{2}) \quad (s_{2}^{2}, a_{2}^{2}) \oplus R(\tau ^{2}) \quad (s_{3}^{2}, a_{3}^{2}) \oplus R(\tau ^{2}) \quad ... \quad (s_{T}^{2}, a_{T}^{2}) \oplus R(\tau ^{2}) \\
...... \\
(s_{1}^{N}, a_{1}^{N}) \oplus R(\tau ^{N}) \quad (s_{2}^{N}, a_{2}^{N}) \oplus R(\tau ^{N}) \quad (s_{3}^{N}, a_{3}^{N}) \oplus R(\tau ^{N}) \quad ... \quad (s_{T}^{N}, a_{T}^{N}) \oplus R(\tau ^{N}) \\
$$
上面的每一项表示，在每个时刻，Agent观察到环境状态为$s_{t}$，如在自动驾驶中就是屏幕的画面，这个就作为监督学习中的样本数据，而此时Agent采取的行动$a_{t}$，就是监督学习中的Ground Truth标签，与监督学习唯一的不同是，我们除了有样本和标签之外，我们还有一个这局游戏最终累积奖励值，作为当看到画面$s_{t}$采取行动$a_{t}$的好坏程度。
我们的目的是求每局奖励的期望值$\bar{R}(\tau)$的最大值，根据高等数学中的相关知识，我们可以通过对$\bar{R}(\tau)$的求导来实现。
## 梯度上升算法
我们定义代价函数为：
$$
\bar{R}_{\theta}(\tau)=\sum_{\tau} R(\tau) p_{\theta}(\tau)
$$
我们对代价函数值进行求导：
$$
\nabla_{\theta} \bar{R}(\tau)=\frac{1}{N}\sum_{n=1}^{N}\sum_{t=1}^{T_{n}} R(\tau ^{n}) \nabla _{\theta} p_{\theta}(a_{t}^{n} | s_{t}^{t})
$$
我们用梯度上升算法求出极值：
$$
\theta = \theta + \alpha \nabla _{\theta} \bar{R}(\tau)
$$

## 代码实现
以上是数学理论上的推导，讲解当前比较流行的Poliyc Gradient算法的数学原理。然而数学原理不等于代码实现，在本节中我们将讨论代码实现的方法和原理。

## 实用技巧
上面的方法，在实际应用，还可以添加如下技巧来提高性能。